In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
from IPython import display
display.clear_output()

In [ ]:
classes={'car': 0, 'van': 1, 'others': 2, 'bus': 3}
classs_id={b:a for (a,b) in classes.items()}

In [ ]:
!mkdir -p dataset/train/images
!mkdir -p dataset/train/labels

In [ ]:
!mkdir -p dataset/valid/images
!mkdir -p dataset/valid/labels

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Make sure to upload this in your Hacktech folder in gdrive: https://drive.google.com/drive/u/1/folders/1X-gCnYc2KJ_txs4xfAnXln13xi_CZYwM

In [ ]:
def chfname(namee):
    num=int(namee[13:])
    num=num+1
    return str(namee[:13])+f"{num:05d}"

In [ ]:
from PIL import Image
# Open the image
image_path = "/content/gdrive/MyDrive/Hacktech/DETRAC-train-data/Insight-MVT_Annotation_Train/MVI_20011/img00001.jpg"
img = Image.open(image_path)
# Get the width and height
img_width, img_height = img.size
print(f"Image width: {img_width}, Image height: {img_height}")

In [ ]:
def process_xml_file(xml_file, output_directory):
    if os.path.exists(xml_file):
        # Parse the target XML file
        tree = ET.parse(xml_file)
        root = tree.getroot()
        sequence_name = root.attrib['name']
        # Create a target directory for TXT files
        try:
            # Try creating the directory; ignore if it already exists
            os.makedirs(output_directory)
        except:
            pass
        # Change the current working directory to the target directory
        os.chdir(output_directory)
        # Initialize filename
        filename = str(sequence_name) + "_" + 'img00001'
        for frame in root.findall('frame'):
            # Open a text file for writing
            with open(f"{filename}.txt", "w") as f:
                # Iterate through targets in the frame
                for target in frame.findall('target_list/target'):
                    clasas = classes[target.find('attribute').attrib['vehicle_type']]
                    box = target.find('box')
                    left = float(box.attrib['left'])
                    top = float(box.attrib['top'])
                    width = float(box.attrib['width'])
                    height = float(box.attrib['height'])
                    xcen = (left + width / 2) / img_width
                    ycen = (top + height / 2) / img_height
                    norm_width = width / img_width
                    norm_height = height / img_height
                    # Extract speed attribute
                    speed = float(target.find('attribute').attrib['speed'])
                    # Write normalized target information to the file
                    content = f"{clasas} {xcen} {ycen} {norm_width} {norm_height} {speed}\n"
                    f.write(content)
            # Update filename for the next iteration
            filename = chfname(filename)
        # Change the current working directory back to the original directory
        os.chdir(origin)
    else:
        print(f"The specified XML file '{xml_file}' does not exist.")


In [ ]:
# Define common variables
origin = os.getcwd()

In [ ]:
def copy_and_rename_file(source_path, destination_path, new_filename):
    try:
        # Copy the file
        shutil.copy(source_path, destination_path)

        # Create the new file path with the desired filename
        new_file_path = f"{destination_path}/{new_filename}"

        # Rename the copied file
        shutil.move(f"{destination_path}/{source_path.split('/')[-1]}", new_file_path)

    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
xml_file_source="/content/gdrive/MyDrive/Hacktech/DETRAC-Train-Annotations-XML/DETRAC-Train-Annotations-XML"
train_image_source='/content/gdrive/MyDrive/Hacktech/DETRAC-train-data/Insight-MVT_Annotation_Train'
def preprocess(lis,label_path,image_path):
  for i in lis:
    process_xml_file(os.path.join(xml_file_source,str(i)+'.xml'),label_path)
    for j in os.listdir(os.path.join(train_image_source,i)):
       copy_and_rename_file(os.path.join(train_image_source,i,j),image_path,str(i)+"_"+str(j))

In [ ]:
import subprocess

# Run the ls command and capture the output
output = subprocess.check_output(["ls", "/content/gdrive/MyDrive/Hacktech/DETRAC-train-data/Insight-MVT_Annotation_Train"]).decode("utf-8")

# Split the output into lines and store them in a list
file_list = output.splitlines()

# Now you have the list of files
print(file_list)


In [ ]:
train = file_list[30:]
E = "/content/dataset/train/images"
lab ="/content/dataset/train/labels"

In [ ]:
preprocess(train,lab,E)

In [ ]:
# train = ["MVI_20011","MVI_20012","MVI_63562","MVI_63561","MVI_63561","MVI_63554","MVI_41073","MVI_41063","MVI_39861","MVI_39781","MVI_40192","MVI_40191","MVI_39811","MVI_20034","MVI_63521","MVI_63554","MVI_63562","MVI_41063"]
# valid = ["MVI_20052","MVI_20033","MVI_20034","MVI_20063","MVI_39781","MVI_40751","MVI_63521","MVI_40211","MVI_20035","MVI_41073"]
# E = "/content/Hacktech/Speed_estimation/ultralytics/yolo/v8/detect/dataset/train/images"
# lab ="/content/Hacktech/Speed_estimation/ultralytics/yolo/v8/detect/dataset/train/labels"
# vE = "/content/Hacktech/Speed_estimation/ultralytics/yolo/v8/detect/dataset/valid/images"
# vlab ="/content/Hacktech/Speed_estimation/ultralytics/yolo/v8/detect/dataset/valid/labels"

In [ ]:
import os

# Define the path to the directory you want to create
directory = '/content/data'

# Create the directory if it doesn't exist
if not os.path.exists(directory):
    os.makedirs(directory)


In [ ]:
import os

def combine_files_by_prefix(directory):
    # Dictionary to hold content of files with same prefix
    files_dict = {}

    # Iterate over files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            prefix = filename[:10]  # Extract first 10 characters as prefix
            filepath = os.path.join(directory, filename)

            # Add file content to dictionary, creating an entry if it doesn't exist
            if prefix not in files_dict:
                files_dict[prefix] = ''
            with open(filepath, 'r') as file:
                files_dict[prefix] += file.read() + '\n'

    # Write combined content to files with corresponding prefix
    for prefix, content in files_dict.items():
        combined_filename = os.path.join('/content/data', f"{prefix}.txt")
        with open(combined_filename, 'w') as combined_file:
            combined_file.write(content)

# Example usage:
directory = '/content/dataset/train/labels'
combine_files_by_prefix(directory)


In [ ]:
import shutil

# Define the path to the folder you want to download
folder_path = '/content/data'

# Define the path for the zip file
zip_file_path = '/content/data.zip'

# Compress the folder into a zip file
shutil.make_archive('/content/data', 'zip', folder_path)

# Download the zip file
from google.colab import files
files.download(zip_file_path)
